In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [52]:
from pandas import Series
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from catboost import CatBoost,CatBoostClassifier, Pool
from catboost.utils import get_roc_curve

from sklearn.metrics import accuracy_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

RANDOM_SEED = 40
import warnings; warnings.simplefilter('ignore')

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [6]:
train.info()

In [7]:
test.info()

In [8]:
sample_submission.info()

In [9]:
# Хочу объединить оба датасета в один: 
train['sample'] = 1
test['sample'] = 0
test['default'] = 0

data = test.append(train, sort = False).reset_index(drop = True)
data

In [10]:
data.info()

**Описание полей:**   
client_id - идентификатор клиента  
education - уровень образования  
sex - пол заемщика  
age - возраст заемщика  
car - флаг наличия автомобиля  
car_type - флаг автомобиля иномарки  
decline_app_cnt - количество отказанных прошлых заявок  
good_work - флаг наличия “хорошей” работы  
bki_request_cnt - количество запросов в БКИ  
home_address - категоризатор домашнего адреса  
work_address - категоризатор рабочего адреса  
income - доход заемщика  
foreign_passport - наличие загранпаспорта  
sna - связь заемщика с клиентами банка  
first_time - давность наличия информации о заемщике  
score_bki - скоринговый балл по данным из БКИ
region_rating - рейтинг региона
app_date - дата подачи заявки
default - флаг дефолта по кредиту

**1. Предобработка данных**

In [11]:
data['education'].value_counts().plot.barh()

In [12]:
data['education'].value_counts()

In [13]:
data['education'] = data['education'].fillna('SCH')
data['education'].value_counts()

In [16]:
data['app_date'] = pd.to_datetime(data['app_date'])
data['app_date'] = data['app_date'].apply(lambda x:x.month)

In [17]:
data['app_date']

Обработка признаков.
План
1. Сделаем три категории: числовые, бинарные, категориальные
2. Найдем те, которые имеют больший вес
3. Представим их в виде, понятном алгоритму

In [18]:
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = ['education', 'home_address', 'work_address']
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt','app_date']

In [19]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))

axes[0,0].hist(data['age'], bins = 50)
axes[0,0].set_title("age")

axes[0,1].hist(data['decline_app_cnt'], bins = 50, range=(0,10))
axes[0,1].set_title("decline_app_cnt")

axes[1,0].hist(data['bki_request_cnt'], bins = 50,  range=(0,20))
axes[1,0].set_title("bki_request_cnt")

axes[1,1].hist(data['income'], bins = 50)
axes[1,1].set_title("income")

Тут мы видим, что все хвосты смещены, поэтому сделаем распределение переменных более нормальным -- с помощью логарифмирования

In [20]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna().apply(lambda x:np.log(x+0.1)), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [21]:
fig, axes = plt.subplots(2, 3, figsize = (15,15))
plt.subplots_adjust(wspace = 0.5)
axes = axes.flatten()

for i in range(len(num_cols)):
    sns.boxplot(x = 'default', y = num_cols[i], data = data, orient = 'v', ax = axes[i])

Кажется, это был не очень нужный шаг, но вообще интересный:   
мы знаем, что дефолтные клиенты в целом младше, количество их отмененных заявок и запросов в БКИ больше,а их доход ниже.

Этап корреляции для числовых параметров

In [22]:
sns.set(font_scale = 1)
fig, axis = plt.subplots(figsize=(7,7))
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1, square=True, annot=True,
            fmt=".2f")

Кажется, что все параметры достаточно независимы. Прекрасно. Пойдем в подробности и сделаем оценку значимости непрерывных переменных

In [23]:
imp_num = Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Каеф. Самый значимый признак из числовых -- это количество отказанных прошлых заявок

**Категориальные признаки**  
Преобразуем их в числа

In [24]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(data['sex']))
print(dict(enumerate(label_encoder.classes_)))

In [25]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(data['education']))
print(dict(enumerate(label_encoder.classes_)))

In [28]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

data['education'] = label_encoder.fit_transform(data['education'])    
# убедимся в преобразовании    
data.head()

In [29]:
# Теперь определяем значимость категориальных признаков

imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [30]:
# Преобразуем категориальные признаки в численные:
data = pd.get_dummies(data,columns = cat_cols)

In [31]:
# Стандартизируем числовые переменные
data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]),columns = num_cols)

In [ ]:
#Подготовка данных
#X_cat = OneHotEncoder(sparse = False).fit_transform(data[cat_cols].values)
#X_cat.shape

In [32]:
data

In [ ]:
# Объединяем
#X = np.hstack([X_num, data[bin_cols].values, X_cat])
#Y = data['default'].values

**4.Приступаем к обучению модели:**  
1. Делим датасет на train и test
2. Запускаем логистическую регрессию
3. Анализируем результаты

In [33]:
# Разделим обратно на train и test
train = data.query('sample == 1').drop(['sample'], axis=1)
test = data.query('sample == 0').drop(['sample'], axis=1)

In [34]:
# удалим для X целевую переменную и client_id
X = train.drop(columns = ['default', 'client_id'])
Y = train['default']

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)

**Обучаем модель**

In [37]:
# Построим модель логистической регресии с параметрами "по умолчанию"
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

In [38]:
probs = model.predict_proba(X_test)
probs = probs[:,1]

fpr, tpr, threshold = roc_curve(Y_test, probs)
roc_auc = roc_auc_score(Y_test, probs)

plt.figure()
fig, ax = plt.subplots(figsize=(10,5))
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
ax.set_title('Logistic Regression ROC AUC = %0.3f'%roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

**Оценим точность**

In [39]:
print('accuracy_score: {}'.format(np.round(accuracy_score(Y_test, Y_pred), 4)))
print('f1_score: {}'.format(np.round(f1_score(Y_test, Y_pred), 4)))
print('recall_score: {}'.format(np.round(recall_score(Y_test, Y_pred), 4)))
print('MSE: {}'.format(np.round(mean_squared_error(Y_test, Y_pred), 4)))

In [40]:
confusion_matrix(Y_test, Y_pred)

In [42]:
# Визуализация confusion matrix:
sns.set_context(context='paper', font_scale=2, rc=None)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                confusion_matrix(Y_test, Y_pred).flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
ax = sns.heatmap(confusion_matrix(Y_test, Y_pred), annot=labels, fmt='', cmap='Greens')
ax.set(title = 'Confusion matrix')
plt.show()

Кажется, что модель довольно точна, но она выдает кредиты, которые впоследствии дефолтнутся. Что ж, надо бы подключить работу с гиперпараметрами, но сил у меня уже нет

In [60]:
X_test2 = test.drop(columns = ['default', 'client_id'])
y_probs = model.predict_proba(X_test2)[:,1]
test['default'] = y_probs
submission = test[['client_id','default']]
display(submission.sample(10))
display(submission.shape)

In [61]:
submission.to_csv('submission.csv', index=False)